In [1]:
import helper_functions as hf
import cv2
# import tensorflow.contrib.tensorrt as trt
import tensorflow as tf
import numpy as np

In [2]:
trt_fname = 'ssd_inception_v2_coco_trt.pb'           # the model after optimization using tensorrt

In [3]:
trt_graph = hf.get_frozen_graph(trt_fname)

input_names = ['image_tensor']

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)
tf.import_graph_def(trt_graph, name='')

tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')
tf_input.shape.as_list()

Instructions for updating:
Use tf.gfile.GFile.



[1, None, None, 3]

In [ ]:
def preprocess(image, contrast_factor=0.85, intensity_thresh=200):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR) 

    """Changing Contrast"""
    if image.std() > 45.0:
        image = np.array(image * contrast_factor, dtype=np.uint8)
    
    """Changing Brightness"""
    _, image = cv2.threshold(image, intensity_thresh, 255, cv2.THRESH_TRUNC)     # truncates the image brightness
    
    return image

In [7]:
"""Parameters to Change"""
score_threshold = 0.5
width, height, fps = 2560, 720, 25

cap = cv2.VideoCapture(0)                   # taking image input from a camera
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)    # setting the width and height of the camera
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

while True :
    retval, image = cap.read()
    
    if not retval:
        break
    
    left_right_image = np.split(image, 2, axis=1)          # dividing the stereo feed across the width
    image = left_right_image[1]                            # taking only one side of the feed as input
    
    # Detect Here...
    scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections],
                                                         feed_dict={tf_input: image[None, ...]})
    boxes = boxes[0]
    scores = scores[0]
    classes = classes[0]
    num_detections = int(num_detections[0])

    """Thresholding"""
    if scores[0] > score_threshold:
        print('Bump :(')
#         hf.visualize_detection(image, num_detections, classes, boxes, scores)
    else:
        print('No Bump :)')
    
#     cv2.imshow("Detection", image)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
x, y